# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd

# Carregar a base
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Número de registros totais
total_registros = sinasc.shape[0]

# Número de registros únicos (não duplicados)
total_registros_unicos = sinasc.drop_duplicates().shape[0]

# Verificar se há duplicados
ha_duplicados = total_registros != total_registros_unicos

print(f"Total de registros: {total_registros}")
print(f"Total de registros únicos: {total_registros_unicos}")
print(f"Há registros duplicados? {'Sim' if ha_duplicados else 'Não'}")


Total de registros: 27028
Total de registros únicos: 27028
Há registros duplicados? Não


In [3]:
# Contar valores missing por coluna
missing_values = sinasc.isnull().sum()

print("Valores missing por variável:")
print(missing_values)


Valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [5]:
# Selecionar as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 
                     'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionada = sinasc[colunas_interesse]

# Contar valores missing nas colunas de interesse
missing_values_interesse = sinasc_selecionada.isnull().sum()

print("Valores missing nas colunas de interesse:")
print(missing_values_interesse)


Valores missing nas colunas de interesse:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [7]:
# Remover registros com APGAR5 não preenchido
sinasc_selecionada = sinasc_selecionada.dropna(subset=['APGAR5'])

# Recontar número de linhas e valores missing
linhas_restantes = sinasc_selecionada.shape[0]
missing_values_pos_remocao = sinasc_selecionada.isnull().sum()

print(f"Total de registros após remoção de APGAR5 não preenchido: {linhas_restantes}")
print("Valores missing por variável após remoção de APGAR5 não preenchido:")
print(missing_values_pos_remocao)


Total de registros após remoção de APGAR5 não preenchido: 26925
Valores missing por variável após remoção de APGAR5 não preenchido:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [13]:
# Substituir valores faltantes ou 'não preenchido' por 9
sinasc_selecionada['ESTCIVMAE'] = sinasc_selecionada['ESTCIVMAE'].fillna(9)
sinasc_selecionada['CONSULTAS'] = sinasc_selecionada['CONSULTAS'].fillna(9)


In [11]:
# Substituir valores faltantes em QTDFILVIVO por zero
sinasc_selecionada['QTDFILVIVO'] = sinasc_selecionada['QTDFILVIVO'].fillna(0)


1ª - as variáveis quantitativas, QTDFILVIVO e IDADEMAE, foram tratados os valores como zero e média, consecutivamente. Pois seria o mis adequado para classificar os filhos e não há porque entender que a mãe não tenha idade, podendo gerar uma discrepância grande na hora de analisar essas informações.

2ª - Os valores qualitativos foram tratados como não informado, dar entender que é a escolha mais adequada. 
SCMAE: Valores ausentes foram preenchidos com "Não informado", pois não seria adequado estimar.

GESTACAO: evita suposições sobre o tempo de gestação, que pode variar significativamente.

GRAVIDEZ: classifica se a gravidez foi única ou múltipla. Valores ausentes foram preenchidos com "Não informado" para manter consistência sem introduzir vieses.

In [15]:
# Substituir valores faltantes nas categóricas por "Não informado"
sinasc_selecionada['LOCNASC'] = sinasc_selecionada['LOCNASC'].fillna('Não informado')
sinasc_selecionada['ESCMAE'] = sinasc_selecionada['ESCMAE'].fillna('Não informado')
sinasc_selecionada['GESTACAO'] = sinasc_selecionada['GESTACAO'].fillna('Não informado')
sinasc_selecionada['GRAVIDEZ'] = sinasc_selecionada['GRAVIDEZ'].fillna('Não informado')

# Substituir valores faltantes em IDADEMAE pela média
media_idademae = sinasc_selecionada['IDADEMAE'].mean()
sinasc_selecionada['IDADEMAE'] = sinasc_selecionada['IDADEMAE'].fillna(media_idademae)


In [17]:
# Verificar se ainda há valores missing
missing_final = sinasc_selecionada.isnull().sum()

print("Valores missing finais por variável:")
print(missing_final)


Valores missing finais por variável:
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [19]:
# Criar a categorização para APGAR5
sinasc_selecionada['apgar5_categoria'] = pd.cut(
    sinasc_selecionada['APGAR5'],
    bins=[-1, 3, 5, 7, 10],  # Intervalos definidos
    labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'],  # Categorias
    right=True  # Intervalo fechado à direita
)

# Calcular as frequências da categorização
frequencias_apgar5 = sinasc_selecionada['apgar5_categoria'].value_counts()

print("Frequências da categorização de APGAR5:")
print(frequencias_apgar5)


Frequências da categorização de APGAR5:
apgar5_categoria
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [23]:
# renomeando todas as colunas, aplicando a transformação para o formato snake_case.
sinasc_selecionada.rename(
    columns={
        'LOCNASC': 'loc_nasc',
        'IDADEMAE': 'idade_mae',
        'ESTCIVMAE': 'est_civ_mae',
        'ESCMAE': 'esc_mae',
        'QTDFILVIVO': 'qtd_fil_vivo',
        'GESTACAO': 'gestacao',
        'GRAVIDEZ': 'gravidez',
        'CONSULTAS': 'consultas',
        'APGAR5': 'apgar5',
        'apgar5_categoria': 'apgar5_categoria'
    },
    inplace=True
)

print("Colunas após renomeação:")
print(sinasc_selecionada.columns)


Colunas após renomeação:
Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_categoria'],
      dtype='object')
